# Equal-Weight S&P 500 Index Fund

In [2]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter # for well formatted excel files from python
from secrets import IEX_CLOUD_API_TOKEN # acquiring API Token

## Preparing data

In [3]:
# the excel file I used was synced with the IEX Sandbox API data so I had to drop some stocks in order to make this program work

stocks = pd.read_csv("sp_500_stocks.csv")
stocks = stocks.drop([135], axis=0)
stocks = stocks.loc[(stocks["Ticker"] != "HFC")]
stocks = stocks.loc[(stocks["Ticker"] != "VIAC")]
stocks = stocks.loc[(stocks["Ticker"] != "WLTW")]
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Populating our Pandas DataFrame with our stocks

In [4]:
# We'll be using Batch API Calls as calling for 500 stocks takes a really long time
# since we can only get data of 100 stocks per batch api call, we are breaking our data down into groups of 100 stocks each at the most

def breakdown(array, length):
    for i in range(0, len(array), length):
        yield array[i:i+length]
        
broken_down = list(breakdown(stocks["Ticker"], 100))
broken_down

[0         A
 1       AAL
 2       AAP
 3      AAPL
 4      ABBV
       ...  
 95     CINF
 96       CL
 97      CLX
 98      CMA
 99    CMCSA
 Name: Ticker, Length: 100, dtype: object,
 100     CME
 101     CMG
 102     CMI
 103     CMS
 104     CNC
        ... 
 196     FTV
 197      GD
 198      GE
 199    GILD
 200     GIS
 Name: Ticker, Length: 100, dtype: object,
 201       GL
 202      GLW
 203       GM
 204     GOOG
 205    GOOGL
        ...  
 297      MAS
 298      MCD
 299     MCHP
 300      MCK
 301      MCO
 Name: Ticker, Length: 100, dtype: object,
 302    MDLZ
 303     MDT
 304     MET
 305     MGM
 306     MHK
        ... 
 397      RL
 398     RMD
 399     ROK
 400     ROL
 401     ROP
 Name: Ticker, Length: 100, dtype: object,
 402    ROST
 403     RSG
 404     RTX
 405    SBAC
 406    SBUX
        ... 
 499     XYL
 500     YUM
 501     ZBH
 502    ZBRA
 503    ZION
 Name: Ticker, Length: 100, dtype: object,
 504    ZTS
 Name: Ticker, dtype: object]

In [5]:
cols = ["Ticker", "Market Cap", "Stock Price", "No. of Shares to Buy"]
updated_stocks = pd.DataFrame(columns = cols)
stock_strings = []

for i in range(len(broken_down)):
    stock_strings.append(','.join(broken_down[i]))
    
stock_strings

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA',
 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD,GIS',
 'GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

In [6]:
for stock_string in stock_strings:
    batch_api_call = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={stock_string}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call).json()
    for string in stock_string.split(","):
        updated_stocks = updated_stocks.append(
            pd.Series(
                [
                    string,
                    data[string]["quote"]["marketCap"],
                    data[string]["quote"]["latestPrice"],
                    "N/A"
                ], 
                index=cols
            ), 
            ignore_index=True
        )

updated_stocks

,Ticker,Market Cap,Stock Price,No. of Shares to Buy
0,A,41147890236,140.330,N/A
1,AAL,9778971551,14.745,N/A
2,AAP,12357144296,197.917,N/A
3,AAPL,2658184046707,168.650,N/A
4,ABBV,260323580587,148.425,N/A
...,...,...,...,...
496,YUM,35695545436,123.800,N/A
497,ZBH,24447362892,117.040,N/A
498,ZBRA,17974981235,330.140,N/A
499,ZION,8400717916,53.860,N/A


## Calculating the Number of Shares to Buy

In [7]:
portfolio_size = input("Enter the value(numerical) of your investment portfolio here: ")

try:
    value = float(portfolio_size)
except ValueError:
    print("Incorrect input. Please make sure you enter a numerical value!")
    portfolio_size = input("Enter the value(numerical) of your investment portfolio here: ")
    value = float(portfolio_size)

In [8]:
position_size = value / len(updated_stocks.index)
print(f"The position size is {position_size}")

for i in range(len(updated_stocks.index)):
    updated_stocks.loc[i, "No. of Shares to Buy"] = position_size // updated_stocks.loc[i, "Stock Price"]
    
updated_stocks

The position size is 1996007.9840319362


,Ticker,Market Cap,Stock Price,No. of Shares to Buy
0,A,41147890236,140.330,14223.0
1,AAL,9778971551,14.745,135368.0
2,AAP,12357144296,197.917,10085.0
3,AAPL,2658184046707,168.650,11835.0
4,ABBV,260323580587,148.425,13447.0
...,...,...,...,...
496,YUM,35695545436,123.800,16122.0
497,ZBH,24447362892,117.040,17054.0
498,ZBRA,17974981235,330.140,6045.0
499,ZION,8400717916,53.860,37059.0


## Formatting Our Excel Output

In [9]:
writer = pd.ExcelWriter("recommended_trades.xlsx", engine="xlsxwriter")
updated_stocks.to_excel(writer, "Recommended Trades", index=False)

### Creating the formats

In [10]:
bg = "#0a0a23"
text = "#ffffff"

string_format = writer.book.add_format({
    "font_color": text,
    "bg_color": bg,
    "border": 1 # solid border around each cell
})

dollar_format = writer.book.add_format({
    "num_format": "$0.00", # every dollar value will be displayed correct to 2 decimal places
    "font_color": text,
    "bg_color": bg,
    "border": 1 # solid border around each cell
})

integer_format = writer.book.add_format({
    "num_format": "0", # every integer will be displayed as a whole number
    "font_color": text,
    "bg_color": bg,
    "border": 1 # solid border around each cell
})

### Applying the Formats

Sample Code:

```python
writer.sheets['Recommended Trades'].set_column('B:B', # apply the format to column B
                     18, # column width of 18 pixels
                     string_template # applies the previously-built format 'string_template' to the column
                     )
```

In [11]:
column_format = {
    "A": ["Ticker", string_format],
    "B": ["Market Cap", dollar_format],
    "C": ["Stock Price", dollar_format],
    "D": ["No. of Shares to Buy", integer_format]
}

for column in column_format:
    writer.sheets["Recommended Trades"].set_column(f"{column}:{column}", 18, column_format[column][1])
    writer.sheets["Recommended Trades"].write(f"{column}1", column_format[column][0], column_format[column][1])

In [12]:
writer.save()